In [1]:
# train_model.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# 1. Veriyi Yükle (İlk satır başlık değil, açıklama olduğu için header=1)
df = pd.read_csv("data-export.csv", header=1)

# Sütun isimlerini sadeleştirme
df.columns = [
    'Channel', 'DateHour', 'Users', 'Sessions', 'EngagedSessions',
    'AvgEngagementTime', 'EngagedSessionsPerUser', 'EventsPerSession',
    'EngagementRate', 'EventCount'
]

# 2. Tarih ve Saat İşlemleri
# DateHour formatı: YYYYMMDDHH (ör: 2024041623)
df['DateTime'] = pd.to_datetime(df['DateHour'], format='%Y%m%d%H')
df['Hour'] = df['DateTime'].dt.hour
df['DayOfWeek'] = df['DateTime'].dt.dayofweek  # 0: Pazartesi, 6: Pazar
df['DayOfMonth'] = df['DateTime'].dt.day

# 3. Model Hazırlığı
# Hedef: Sessions (Oturum Sayısı)
# Girdiler: Channel, Hour, DayOfWeek
X = df[['Channel', 'Hour', 'DayOfWeek']]
y = df['Sessions']

# Veriyi ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Pipeline (Kategorik Dönüşüm + Model)
categorical_features = ['Channel']
numeric_features = ['Hour', 'DayOfWeek']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 5. Eğitme
print("Trafik tahmin modeli eğitiliyor...")
model.fit(X_train, y_train)

# 6. Değerlendirme
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"Model RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.4f}")

# 7. Kaydetme
joblib.dump(model, 'traffic_model.pkl')
print("Model 'traffic_model.pkl' olarak kaydedildi.")

Trafik tahmin modeli eğitiliyor...
Model RMSE: 18.41
R2 Score: 0.7823
Model 'traffic_model.pkl' olarak kaydedildi.
